In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

# Load the data
data = pd.read_csv("C:/Users/HP/Desktop/NSSO68.csv")

# Display the first few rows and column names to identify correct column names
print(data.head())
print(data.columns)

# Replace 'dependent_variable', 'independent_variable_1', 'independent_variable_2', etc.
# with actual column names from your dataset.

# Example column names (replace these with actual names from your dataset)
dependent_var = 'income'  # Dependent variable
independent_vars = ['age', 'education', 'experience']  # Independent variables

# Ensure the dependent variable is numeric
data[dependent_var] = pd.to_numeric(data[dependent_var], errors='coerce')

# Ensure the independent variables are numeric
for var in independent_vars:
    data[var] = pd.to_numeric(data[var], errors='coerce')

# Drop rows with missing values
data = data.dropna()

# Define dependent and independent variables
y = data[dependent_var]
X = data[independent_vars]

# Add a constant to the independent variables
X = sm.add_constant(X)

class TobitModel(GenericLikelihoodModel):
    def __init__(self, endog, exog, left_censoring=0, **kwds):
        self.left_censoring = left_censoring
        super(TobitModel, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self, params):
        endog = self.endog
        exog = self.exog
        left_censoring = self.left_censoring

        beta = params[:-1]
        sigma = params[-1]
        
        XB = np.dot(exog, beta)
        censored = endog <= left_censoring
        uncensored = endog > left_censoring

        llf_censored = np.log(1 - sm.distributions.norm.cdf(XB[censored] / sigma))
        llf_uncensored = -0.5 * (np.log(2 * np.pi) + np.log(sigma ** 2) + ((endog[uncensored] - XB[uncensored]) / sigma) ** 2)

        return -np.sum(np.concatenate([llf_censored, llf_uncensored]))

# Fit the Tobit model
model = TobitModel(y, X)
results = model.fit()

# Print the summary of the model
print(results.summary())

# If you wish to calculate marginal effects, you can extend the code accordingly.


C:\Users\HP\AppData\Local\Temp\ipykernel_22688\1060529911.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:/Users/HP/Desktop/NSSO68.csv")


   slno                                 grp  Round_Centre  FSU_number  Round  \
0     1  40999999999999992652495293775872.0             1       41000     68   
1     2  40999999999999992652495293775872.0             1       41000     68   
2     3  40999999999999992652495293775872.0             1       41000     68   
3     4  40999999999999992652495293775872.0             1       41000     68   
4     5  40999999999999992652495293775872.0             1       41000     68   

   Schedule_Number  Sample  Sector  state  State_Region  ...  pickle_v  \
0               10       1       2     24           242  ...       0.0   
1               10       1       2     24           242  ...       0.0   
2               10       1       2     24           242  ...       0.0   
3               10       1       2     24           242  ...       0.0   
4               10       1       2     24           242  ...       0.0   

   sauce_jam_v  Othrprocessed_v  Beveragestotal_v  foodtotal_v  foodtotal_

KeyError: 'income'